In [1]:
%load_ext rpy2.ipython

In [2]:
from pandas import read_csv, DataFrame, Series
import statsmodels.api as sm
import rpy2.robjects as R
from rpy2.robjects.packages import importr
from rpy2.robjects import r, pandas2ri
from pandas import date_range
import psycopg2
from psycopg2.extras import DictCursor
from matplotlib import pyplot
from datetime import datetime, timezone


pandas2ri.activate()

C:\Users\A.Litvinau\AppData\Local\Programs\Python\Python36-32\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [22]:
dataset = read_csv('dataset.csv', index_col=['date'], parse_dates=['date'], dayfirst=True, usecols=['date', 'weighted_average', 'high', 'low', 'open', 'close'])

In [23]:
dataset

,high,low,open,close,weighted_average
date,,,,,
2016-07-19 08:35:00,0.989950,0.989950,0.989950,0.989950,0.989950
2016-07-19 08:40:00,0.989950,0.250000,0.989950,0.250000,0.269013
2016-07-19 08:45:00,0.250000,0.250000,0.250000,0.250000,0.250000
2016-07-19 08:50:00,0.970000,0.950000,0.950000,0.970000,0.966364
2016-07-19 08:55:00,0.970000,0.970000,0.970000,0.970000,0.970000
2016-07-19 09:00:00,0.970000,0.970000,0.970000,0.970000,0.970000
2016-07-19 09:05:00,0.970000,0.970000,0.970000,0.970000,0.970000
2016-07-19 09:10:00,0.970000,0.970000,0.970000,0.970000,0.970000
2016-07-19 09:15:00,0.550000,0.550000,0.550000,0.550000,0.550000


In [24]:
dataset.plot(figsize=(12,6))
pyplot.show()